In this notebook, I'll work through [this guide by Simon Boehm on fast matmul](https://siboehm.com/articles/22/CUDA-MMM)

For simplicity, I'll use cuda from numba. According to [its documentation](https://numba.readthedocs.io/en/stable/cuda/overview.html#missing-cuda-features) it misses some cuda feature, but these features are not required for this guide:
- [dynamic parallelism](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#cuda-dynamic-parallelism): call kernels function from other kernel functions
- [texture memory](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#texture-and-surface-memory): a special memory for graphics applications

Let's start!

----

### Prep 1 - Using numba cuda

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from fastcore.basics import tuplify

#os.environ['NUMBA_ENABLE_CUDASIM']='1'  # enables simulator 
os.environ['CUDA_LAUNCH_BLOCKING']='1'

from numba import cuda, float32
from util import to_d, to_h, array_like, cdiv

np.set_printoptions(formatter={'float': lambda x: f'{x:.1f}'}, linewidth=200)

dtype = 'float32'

In [2]:
@cuda.jit
def f(a, b, c):
    tid = cuda.grid(1) # like threadIdx.x + (blockIdx.x * blockDim.x)
    if tid >= len(c): return
    c[tid] = a[tid] + b[tid]

In [3]:
n = 30
a = to_d(np.ones(n, dtype=dtype))
b = to_d(np.ones(n, dtype=dtype))
c = array_like(a)

In [4]:
nthreads = 8
nblocks = (len(a) // nthreads) + 1
f[nblocks, nthreads](a, b, c)
to_h(c)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], dtype=float32)

### Prep 2 - Profiling numba cuda

There are 2 profiling options:
1. **Simple:** This measures the cuda runtime of a kernel
2. **Detailled:** This performs a detailled analysis of the kernel run, and gives hints how the kernel could be improved

I can do **simple** profiling in a notebook, which I'll do.

For **detailled** profiling, I can use 'nsight compute' (a program by nvidia; short name 'ncu'), which requires the code to be in a separate file, so can't be done in jupyter.

I can then use `ncu --set full --import-source yes -f -o <output_location> --page details python3 <python_filename>` for profiling. Then I can either look a the the console output for a quick analysis, or load the output into the nsight compute ui program (which I've downloaded) for more details.

_Note: I could do detailled profiling in jupyter with `%%writefile <py_filename>`, `%run  <py_filename>` and `!ncu ...`, but I don't see the benefit._

In [5]:
from util import measure_runtime

### Kernel 1: Naive Implementation

In [6]:
@cuda.jit()
def matmul_1(a,b,c,m,n,k):
    x,y = cuda.grid(2)
    if x >= m or y >= n: return 
    tmp = np.float32(0)
    for i in range(k): tmp += a[x,i] * b[i,y]
    c[x, y] = tmp

In [7]:
m,k,n = 2,3,4

a = to_d(np.ones((m,k), dtype='float32'))
b = to_d(np.ones((k,n), dtype='float32'))
c = to_d(np.empty((m,n), dtype='float32'))

nthreads = (2,2)
nblocks = cdiv(c.shape, nthreads)
nthreads, nblocks

((2, 2), (1, 2))

In [8]:
matmul_1[nblocks, nthreads](a,b,c,m,n,k)
to_h(c)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([[3.0, 3.0, 3.0, 3.0],
       [3.0, 3.0, 3.0, 3.0]], dtype=float32)

In [9]:
measure_runtime(matmul_1);

Measuring runtime of kernel matmul_1 for m,n,k = 4092,4092,4092, averaging over 3 runs.


STAGE:2024-05-05 14:15:16 117062:117062 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-05-05 14:15:20 117062:117062 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-05-05 14:15:20 117062:117062 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


1.242s


### Kernel 2: Global Memory Coalescing

We're doing naive matmul: A @ B = C.<br/>
Sizes are (3,4) @ (4,5) = (3,5)

Let's say we have threads 0,1 and 6,7; where 0/1 and 6/7 are consecutive respectively.<br/>
Remember, consecutive threads are (likely) in the same warp and can pool data loads. For simplicity, let's say warps are size 2 (eg threads 0/1 form a warp; threads 6/7 form a warp).<br/>
Also, note the consecutiveness is determined by the cuda runtime; we can't influence it and it doesn't depend on the computation.<br/>
    What we *can* decide is how threads are mapped to computation pieces. A smart choice can save data reads.

Let's assume the input matrices A and B are in row-major ordering, which is very often the case.

To compute C[row, col] we need A[row, :] and B[:, col].
In row-major ordering, A[row, :] is contiguous in memory, while B[:, col] is *not*.

We know gpus read memory in 'bursts'. For simplicity, let's say the burst size is 3.

This means eg if we ask for A[0,0], we get A[0,0:2].

So if we ask for A[0,:] = A[0,0:5] we get A[0,0:3] & A[0,3:6]. Here is the colloquial 'and'. (Also A[0,4:6] don't exist, so will be effectively random).<br/>
-> We need 4 values, but are forced to read 2*3=6.

If we ask for B[:,0] = B[0:5,0] we get B[0,0:3] & B[1,0:3] & B[2,0:3] & B[3,0:3] & B[4,0:3].<br/>
-> We need 4 values, but are forced to read 4*3=12.

So reading a col of B is way more wasteful than reading a row of A.<br/>
Can we smartly assign threads to computation pieces in order to use this waste? Turns out: Yes!

Let's think through 2 options:<br/>

    1) If consecutive threads share the same row (eg warp 0 [consisting of threads 0/1] is assigned to compute C[0,0]/C[0,1]),
        the warp needs values A[0,:] and B[:,0:2].
        Because of bursting, it'll read:
            A[0,0:3]
            A[0,3:6]
            B[0,0:3]
            B[1,0:3]
            B[2,0:3]
            B[3,0:3]
            B[4,0:3]
        Not that waste in B is actually used here, becase the warp needs data from different cols.
        In total, we're 7 reads (each a burst, so of size 3). 

    2) If consecutive threads share the same col (eg warp 0 [consisting of threads 0/1] is assigned to compute C[0,0]/C[1,0]),
        the warp needs values A[0:2,:] and B[:,0].
        Because of bursting, it'll read:
            A[0,0:3]
            A[0,3:6]
            A[0,0:3]
            A[0,3:6]
            B[0,0:3]
            B[1,0:3]
            B[2,0:3]
            B[3,0:3]
            B[4,0:3]
        In total, we're 9 reads (each a burst, so of size 3). 
'

Cuda groups a bunch of 'consecutive' threads into a group, called a 'warp'. This is useful, because threads in a warp can pool their data reads. E.g., if threads 1 and 2 are in the same warp, and thread 1 needs to read data from locations x and y, and thread 2 from location y and z, the warp only needs to do 3 reads (x,y,z) instead of 4 (x,y, y,z)!

Different blocks are completely independent. However, inside a block, cuda has a notion of 'consecutiveness' by assigning a thread_id (tid) to each thread, like so:

`tid = tidx + tidy * bsx + tidz * bsx * bsy`.

Note that to increase `tid` by 1, you'd have to increase `tidx` by 1, not `tidy` or `tidz`. This means threads with consecutive `tidx`s are very likely to be consecutive. If we can make it so threads with consecutive `tidx`s need to read identical (over at least strongly overlapping) data, the pooled data read can save a lot of time.

**So we want consecutive threads to compute the same row!**

In the above naive implementation, 

In [10]:
@cuda.jit()
def matmul_2(a,b,c,m,n,k,bs):
    # we defined blocks of size bs*bs
    x = cuda.blockIdx.x * bs + (cuda.threadIdx.x // bs)
    y = cuda.blockIdx.y * bs + (cuda.threadIdx.x % bs)
    if x>=m or y>=n: return 
    tmp = np.float32(0)
    for i in range(k): tmp += a[x,i] * b[i,y]
    c[x, y] = tmp

In [11]:
m,k,n = 2,3,4

a = to_d(np.ones((m,k), dtype='float32'))
b = to_d(np.ones((k,n), dtype='float32'))
c = to_d(np.empty((m,n), dtype='float32'))

bs = 2
nthreads = bs*bs
nblocks = cdiv(c.shape, (bs,bs))
nthreads, nblocks

(4, (1, 2))

In [12]:
matmul_2[nblocks, nthreads](a,b,c,m,n,k,bs)
to_h(c)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([[3.0, 3.0, 3.0, 3.0],
       [3.0, 3.0, 3.0, 3.0]], dtype=float32)

In [13]:
measure_runtime(
    matmul_2,
    nthreads=32*32,nblocks_fn=lambda outp_shape,nthreads: cdiv(outp_shape, (32,32)),
    kernel_args=[32]
);

Measuring runtime of kernel matmul_2 for m,n,k = 4092,4092,4092, averaging over 3 runs.


STAGE:2024-05-05 14:15:22 117062:117062 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-05-05 14:15:24 117062:117062 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-05-05 14:15:24 117062:117062 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


0.558s


In [24]:
@cuda.jit()
def matmul_2_bs32(a,b,c,m,n,k):
    bs = 32
    # we defined blocks of size bs*bs
    x = cuda.blockIdx.x * bs + (cuda.threadIdx.x // bs)
    y = cuda.blockIdx.y * bs + (cuda.threadIdx.x % bs)
    if x>=m or y>=n: return 
    tmp = np.float32(0)
    for i in range(k): tmp += a[x,i] * b[i,y]
    c[x, y] = tmp

In [25]:
measure_runtime(
    matmul_2_bs32,
    nthreads=32*32,nblocks_fn=lambda outp_shape,nthreads: cdiv(outp_shape, (32,32)),
);

Measuring runtime of kernel matmul_2_bs32 for m,n,k = 4092,4092,4092, averaging over 3 runs.


STAGE:2024-05-05 14:26:01 117062:117062 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-05-05 14:26:03 117062:117062 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-05-05 14:26:03 117062:117062 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


0.558s


In [26]:
@cuda.jit()
def matmul_2__bs32__no_premature_return(a,b,c,m,n,k):
    bs = 32
    # we defined blocks of size bs*bs
    x = cuda.blockIdx.x * bs + (cuda.threadIdx.x // bs)
    y = cuda.blockIdx.y * bs + (cuda.threadIdx.x % bs)
    if (x<m and y<n): 
        tmp = np.float32(0)
        for i in range(k): tmp += a[x,i] * b[i,y]
        c[x, y] = tmp

In [27]:
measure_runtime(
    matmul_2__bs32__no_premature_return,
    nthreads=32*32,nblocks_fn=lambda outp_shape,nthreads: cdiv(outp_shape, (32,32)),
);

Measuring runtime of kernel matmul_2__bs32__no_premature_return for m,n,k = 4092,4092,4092, averaging over 3 runs.


STAGE:2024-05-05 14:38:29 117062:117062 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-05-05 14:38:31 117062:117062 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-05-05 14:38:31 117062:117062 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


0.557s


### Kernel 3: Shared Memory Cache-Blocking

In [14]:
@cuda.jit()
def matmul_3_bs2(a,b,c,m,n,k):
    bs = 2 # shared memory requires compile time constanst, so sadly can't use a bs variable
    # index of current block
    bx,by = cuda.blockIdx.x, cuda.blockIdx.y
    # index of current thread inside block
    tx,ty = cuda.threadIdx.x//bs, cuda.threadIdx.x%bs
    # Declare 2 matrices of shape (bs,bs) in shared mem, they'll hold a piece of a/b.
    sh_a, sh_b = cuda.shared.array((bs,bs), float32), cuda.shared.array((bs,bs), float32)        
    tmp = np.float32(0)
    # split k dimension into chunks of size bs, and iterate tru them
    nk = (k+bs-1)//bs # = cdiv(k,bs)
    for bk in range(nk):
        # copy a piece of a & b into shared mem (if we're not out of bounds), and wait until all threads have done so (otherwise computation below will start with wrong data)
        if (bx*bs+tx<m) and (bk*bs+ty<k) and (bk*bs+tx<k) and (by*bs+ty<n):
            sh_a[tx,ty] = a[bx*bs+tx, bk*bs+ty] # = indexing into grid at [bx,bk] to get block, then indexing into block at [tx,ty] to get thread
            sh_b[tx,ty] = b[bk*bs+tx, by*bs+ty] # = indexing into grid at [bk,by] to get block, then indexing into block at [tx,ty] to get thread
        cuda.syncthreads()
        # compute dot product, and wait until all computation is done (otherwise data copy in next iter will overwrite values we need)
        for i in range(bs):
            if (bx*bs+tx <m) and (bk*bs+i<k) and (by*bs+ty<n):
                tmp += sh_a[tx,i]*sh_b[i,ty]
        cuda.syncthreads()
    # write output
    if bx*bs+tx<m and by*bs+ty<n: c[bx*bs+tx,by*bs+ty] = tmp

In [15]:
m,k,n = 2,3,4

a = to_d(np.ones((m,k), dtype='float32'))
b = to_d(np.ones((k,n), dtype='float32'))
c = to_d(np.empty((m,n), dtype='float32'))

bs = 2
nthreads = bs*bs
nblocks = cdiv(c.shape, (bs,bs))
nthreads, nblocks

(4, (1, 2))

In [16]:
matmul_3_bs2[nblocks, nthreads](a,b,c,m,n,k)
to_h(c)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([[3.0, 3.0, 3.0, 3.0],
       [3.0, 3.0, 3.0, 3.0]], dtype=float32)

In [17]:
@cuda.jit()
def matmul_3_bs32(a,b,c,m,n,k):
    bs = 32
    bx,by = cuda.blockIdx.x, cuda.blockIdx.y
    tx,ty = cuda.threadIdx.x//bs, cuda.threadIdx.x%bs
    sh_a, sh_b = cuda.shared.array((bs,bs), float32), cuda.shared.array((bs,bs), float32)        
    tmp = np.float32(0)
    nk = (k+bs-1)//bs
    for bk in range(nk):
        for i in range(bs):
            if (bx*bs+tx<m) and (bk*bs+i<k) and (by*bs+ty<n):
                sh_a[tx,ty] = a[bx*bs+tx, bk*bs+i ]
                sh_b[tx,ty] = b[bk*bs+i , by*bs+ty]
        cuda.syncthreads()
        for i in range(bs):
            if (bx*bs+tx <m) and (bk*bs+i<k) and (by*bs+ty<n):
                tmp += sh_a[tx,i]*sh_b[i,ty]
        cuda.syncthreads()
    if bx*bs+tx<m and by*bs+ty<n: c[bx*bs+tx,by*bs+ty] = tmp

In [18]:
measure_runtime(
    matmul_3_bs32,
    nthreads=32*32,nblocks_fn=lambda outp_shape,nthreads: cdiv(outp_shape, (32,32))
);

Measuring runtime of kernel matmul_3_bs32 for m,n,k = 4092,4092,4092, averaging over 3 runs.


STAGE:2024-05-05 14:15:27 117062:117062 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-05-05 14:15:31 117062:117062 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-05-05 14:15:31 117062:117062 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


1.133s


In [19]:
profile_log = measure_runtime(
    matmul_3_bs32,
    nthreads=32*32,nblocks_fn=lambda outp_shape,nthreads: cdiv(outp_shape, (32,32)),
    return_log=True
);

Measuring runtime of kernel matmul_3_bs32 for m,n,k = 4092,4092,4092, averaging over 3 runs.


STAGE:2024-05-05 14:15:34 117062:117062 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-05-05 14:15:37 117062:117062 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-05-05 14:15:37 117062:117062 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


1.135s


In [21]:
print(profile_log.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
cudapy::__main__::matmul_3_bs32[abi:v5][abi:cw51cXTL...         0.00%       0.000us         0.00%       0.000us       0.000us        3.406s        98.84%        3.406s        1.135s             3  
                       Memcpy HtoD (Pageable -> Device)         0.00%       0.000us         0.00%       0.000us       0.000us      39.983ms         1.16%      39.983ms      13.328ms             3  
         

### Kernel 4: 1D Blocktiling for Calculating Multiple Results per Thread

In [20]:
raise ValueError()

ValueError: 

### Kernel 5: Increasing Arithmetic Intensity via 2D Blocktiling

### Kernel 6: Vectorize SMEM and GMEM Accesses

### Where are kernels 7 & 8??

### Kernel 9: Autotuning

### Kernel 10: Warptiling

### wip - Kernel 11